# Data cleaning V2

In [1]:
# Modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load DataFrames
Podcast_Train_df = pd.read_csv('Data/train.csv')
Podcast_Test_df = pd.read_csv('Data/test.csv')

In [3]:
# to make the preprocessing at the same time
Podcast_Train_df['is_train'] = 1
Podcast_Test_df['is_train'] = 0

In [4]:
# join the both DataFrames
Podcast_df = pd.concat([Podcast_Train_df,Podcast_Test_df])

In [5]:
# 1x10^6 rows
Podcast_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 249999
Data columns (total 13 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   id                           1000000 non-null  int64  
 1   Podcast_Name                 1000000 non-null  object 
 2   Episode_Title                1000000 non-null  object 
 3   Episode_Length_minutes       884171 non-null   float64
 4   Genre                        1000000 non-null  object 
 5   Host_Popularity_percentage   1000000 non-null  float64
 6   Publication_Day              1000000 non-null  object 
 7   Publication_Time             1000000 non-null  object 
 8   Guest_Popularity_percentage  805138 non-null   float64
 9   Number_of_Ads                999999 non-null   float64
 10  Episode_Sentiment            1000000 non-null  object 
 11  Listening_Time_minutes       750000 non-null   float64
 12  is_train                     1000000 non-null  i

## Cleaning process 
* Episode_Title: Change it  with the number
* Episode_Length_minutes: maximum time 121 minutes
* NaN values by the average of the program day and time of emission.
* Host_Popularity_percentage: maximum 100%
* Guest_Popularity_percentage: maximum 100%
* Number_of_Ads: only allowed [0, 1, 2, 3], other values removed


In [6]:
# "Number_of_Ads" NaN ->0
Podcast_df["Number_of_Ads"] = Podcast_df["Number_of_Ads"].fillna(0)

In [7]:
# Define the list of correct values
correct_values = [0.00, 1.00, 2.00, 3.00]

# Create a boolean mask to identify erroneous values
# Values that are NOT in the correct list 
erroneous_mask = ~Podcast_df["Number_of_Ads"].isin(correct_values) 
Podcast_df.loc[erroneous_mask, "Number_of_Ads"] = 1.00

In [8]:
# Change the Host_Popularity_percentage maximun by 100
threshold_host = 100.00
Podcast_df.loc[Podcast_df['Host_Popularity_percentage']>threshold_host, 'Host_Popularity_percentage']= threshold_host

In [9]:
# Change the Guest_Popularity_percentage maximun by 100
threshold_Guest = 100.00
Podcast_df.loc[Podcast_df['Guest_Popularity_percentage']>threshold_Guest, 'Guest_Popularity_percentage']= threshold_Guest

In [10]:
# Change the Guest_Popularity_percentage maximun by 100
threshold_min = 121.00
Podcast_df.loc[Podcast_df['Episode_Length_minutes']>threshold_min, 'Episode_Length_minutes']= threshold_min

In [11]:
Podcast_df.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes,is_train
count,1000000.000000,884171.000000,1000000.000000,805138.000000,1000000.000000,750000.000000,1000000.000000
mean,499999.500000,64.515301,59.823641,52.225207,1.347722,45.437406,0.750000
std,288675.278932,32.964416,22.874055,28.449038,1.111388,27.138306,0.433013
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000
25%,249999.750000,35.740000,39.370000,28.370000,0.000000,23.178350,0.750000
50%,499999.500000,63.870000,60.020000,53.540000,1.000000,43.379460,1.000000
75%,749999.250000,94.080000,79.490000,76.590000,2.000000,64.811580,1.000000
max,999999.000000,121.000000,100.000000,100.000000,3.000000,119.970000,1.000000


In [12]:
# Change episode number to a number
Podcast_df["Episode_Number"] = Podcast_df["Episode_Title"].str.extract(r"(\d+)").astype(float)
Podcast_df['Guest_Popularity_percentage'] = Podcast_df['Guest_Popularity_percentage'].fillna(0) 

In [13]:
# Drop unused
Podcast_df.drop(columns=["Episode_Title"], inplace=True)

In [15]:
# encode categorical
cat_cols =["Podcast_Name", "Genre", "Publication_Day", "Publication_Time", "Episode_Sentiment"]
for col in cat_cols:
    Podcast_df[col] = Podcast_df[col].astype('category')

In [16]:
Podcast_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 249999
Data columns (total 13 columns):
 #   Column                       Non-Null Count    Dtype   
---  ------                       --------------    -----   
 0   id                           1000000 non-null  int64   
 1   Podcast_Name                 1000000 non-null  category
 2   Episode_Length_minutes       884171 non-null   float64 
 3   Genre                        1000000 non-null  category
 4   Host_Popularity_percentage   1000000 non-null  float64 
 5   Publication_Day              1000000 non-null  category
 6   Publication_Time             1000000 non-null  category
 7   Guest_Popularity_percentage  1000000 non-null  float64 
 8   Number_of_Ads                1000000 non-null  float64 
 9   Episode_Sentiment            1000000 non-null  category
 10  Listening_Time_minutes       750000 non-null   float64 
 11  is_train                     1000000 non-null  int64   
 12  Episode_Number               10000

In [ ]:
for pod in Podcast_df['Podcast_Name'].unique():
    pod_df = Podcast_df[Podcast_df['Podcast_Name'] == pod]
    pod_df.to_csv(f'Data/aux/pod.{pod}.Clean.csv', index=False)

In [17]:
Podcast_df['Podcast_Name'].unique()

['Mystery Matters', 'Joke Junction', 'Study Sessions', 'Digital Digest', 'Mind & Body', ..., 'Laugh Line', 'True Crime Stories', 'Business Insights', 'Fashion Forward', 'Tune Time']
Length: 48
Categories (48, object): ['Athlete's Arena', 'Brain Boost', 'Business Briefs', 'Business Insights', ..., 'True Crime Stories', 'Tune Time', 'Wellness Wave', 'World Watch']

In [21]:
n=0
for podcast in Podcast_df['Podcast_Name'].unique():
    podname_df = Podcast_df[Podcast_df['Podcast_Name'] == podcast]
    i=0  
    for day in ['Sunday' ,'Monday' ,'Tuesday' ,'Wednesday','Thursday','Friday', 'Saturday']:
        pub_day_df = podname_df[podname_df['Publication_Day'] == day ]
        j=0
        for hour in ['Night', 'Morning', 'Afternoon', 'Evening']:
            pub_hour_df = pub_day_df[pub_day_df['Publication_Time'] == hour ]
            pub_hour_df['Episode_Length_minutes'] = pub_hour_df['Episode_Length_minutes'].fillna(pub_hour_df['Episode_Length_minutes'].mean())
            pub_hour_df.to_csv(f'Data/aux/{n}.{i}.{j}.Clean.csv', index=False)
            j += 1
        i += 1
    n += 1


/var/folders/_2/nh5rkkms0rbdwypbjgwb50xh0000gn/T/ipykernel_55322/269019889.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pub_hour_df['Episode_Length_minutes'] = pub_hour_df['Episode_Length_minutes'].fillna(pub_hour_df['Episode_Length_minutes'].mean())
/var/folders/_2/nh5rkkms0rbdwypbjgwb50xh0000gn/T/ipykernel_55322/269019889.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pub_hour_df['Episode_Length_minutes'] = pub_hour_df['Episode_Length_minutes'].fillna(pub_hour_df['Episode_Length_minutes'].me

In [23]:
Podcast_df.columns

Index(['id', 'Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'is_train', 'Episode_Number'],
      dtype='object')